# Imports

In [3]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

from google.colab import widgets
import os
import argparse
from torch import LongTensor, FloatTensor
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd
import cv2 as cv
import re
import string
import unicodedata 
import os
import codecs
import random
import logging
import json
import itertools
import math
import copy
import numpy as np
import time
import sys
import gc
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
from collections import Counter, defaultdict
from easydict import EasyDict as edict
from PIL import Image

%matplotlib inline

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

path_to_folder = '/content/gdrive/My Drive/'
noise_dir = os.path.join(path_to_folder, 'noise')
data_dir = os.path.join(noise_dir, 'ujnn2019')
checkpoint_path = os.path.join(data_dir, 'checkpoint.pkl')
results_path = os.path.join(data_dir, 'test.out')

!mkdir -p checkpoints

# Custom Dataset

In [4]:
def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor

In [5]:
class TrainDataset2(Dataset):
    def __init__(self, clean_dir, distorted_dir, siz, transform=None):
        self.clean_dir = clean_dir
        self.dist_dir = distorted_dir
        self.siz = siz
        self.transform = transform
        
    def __getitem__(self, idx):
        idx = idx + 1
        #clean_img = Image.open(os.path.join(self.clean_dir, f'{idx}.jpg'))
        #dist_img = Image.open(os.path.join(self.dist_dir, f'{idx}.jpg'))
        
        with open(os.path.join(self.dist_dir, f'{idx}.jpg'), 'rb') as f:
            dist_img = Image.open(f)
            transformed_dist = self.transform(dist_img)
        
        with open(os.path.join(self.clean_dir, f'{idx}.jpg'), 'rb') as f:
            clean_img = Image.open(f)
            transformed_clean = self.transform(clean_img)

        return (transformed_clean, transformed_dist)

    def __len__(self):
        return self.siz

class ValidDataset2(Dataset):
    def __init__(self, clean_dir, distorted_dir, siz, transform=None):
        self.clean_dir = clean_dir
        self.dist_dir = distorted_dir
        self.siz = siz
        self.transform = transform
        
    def __getitem__(self, idx):
        idx = idx + (10000 - self.siz) + 1
        #clean_img = Image.open(os.path.join(self.clean_dir, f'{idx}.jpg'))
        #dist_img = Image.open(os.path.join(self.dist_dir, f'{idx}.jpg'))
        
        with open(os.path.join(self.dist_dir, f'{idx}.jpg'), 'rb') as f:
            dist_img = Image.open(f)
            transformed_dist = self.transform(dist_img)
        
        with open(os.path.join(self.clean_dir, f'{idx}.jpg'), 'rb') as f:
            clean_img = Image.open(f)
            transformed_clean = self.transform(clean_img)

        return (transformed_clean, transformed_dist)

    def __len__(self):
        return self.siz
      
class TestDataset2(Dataset):
    def __init__(self, test_dir, transform):
        self.test_dir = test_dir
        self.siz = 400
        self.transform = transform
    
    def __getitem__(self, idx):
        idx = idx + 1
        #test_img = Image.open(os.path.join(test_dir, f'{idx}.jpg'))
        with open(os.path.join(test_dir, f'{idx}.jpg'), 'rb') as f:
            test_img = Image.open(f)
            transformed_test = self.transform(test_img)
        
        return transformed_test
    
    def __len__(self):
        return self.siz

# Full RAM datasets

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, clean, distorted, transform=None):
        self.clean = clean
        self.distorted = distorted
        self.transform = transform
        
    def __getitem__(self, idx):
        clean_ex = self.clean[idx]
        #clean_ex = min_max_normalization(clean_ex, 0, 1)
        dist_ex = self.distorted[idx]
        #dist_ex = min_max_normalization(dist_ex, 0, 1)
        return (clean_ex, dist_ex)

    def __len__(self):
        return self.clean.shape[0]

class ValidDataset(Dataset):
    def __init__(self, clean, distorted, transform=None):
        self.clean = clean
        self.distorted = distorted
        self.transform = transform
        
    def __getitem__(self, idx):
        #clean_ex = self.transform(self.clean[idx])
        #dist_ex = self.transform(self.distorted[idx])
        clean_ex = self.clean[idx]
        #clean_ex = min_max_normalization(clean_ex, 0, 1)
        dist_ex = self.distorted[idx]
        #dist_ex = min_max_normalization(dist_ex, 0, 1)
        return (clean_ex, dist_ex)

    def __len__(self):
        return self.clean.shape[0]

class TestDataset(Dataset):
    def __init__(self, distorted, transform=None):
        self.distorted = distorted
        self.transform = transform
        
    def __getitem__(self, idx):
        #dist_ex = self.transform(self.distorted[idx])
        dist_ex = self.distorted[idx]
        return dist_ex

    def __len__(self):
        return self.distorted.shape[0]

# Utils

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Lambda(lambda tensor: tensor.view(48*48*3))
])

def get_clean_and_distorted_examples():
    clean_dir = os.path.join(data_dir, 'clean')
    dist_dir = os.path.join(data_dir, 'distorted')
    
    clean_files = os.listdir(clean_dir)
    dist_files = os.listdir(dist_dir)
    
    clean_images, dist_images = [], []
    for file_id, filename in enumerate(clean_files):
        #if file_id == 10: break
        if file_id % 10 == 0: print(file_id)
        with open(os.path.join(clean_dir, filename), 'rb') as f:
            clean_img = Image.open(f)
            transformed_clean = transform(clean_img)
            clean_images.append(transformed_clean)
    
    for file_id, filename in enumerate(dist_files):
        if file_id % 10 == 0: print(file_id)
        #if file_id == 10: break
        with open(os.path.join(dist_dir, filename), 'rb') as f:
            dist_img = Image.open(f)
            transformed_dist = transform(dist_img)
            dist_images.append(transformed_dist)
    
    clean_tensor, dist_tensor = torch.stack(clean_images), torch.stack(dist_images)
    return clean_tensor, dist_tensor
    
def get_test_examples():
    test_dir = os.path.join(data_dir, 'test_distorted')
    test_files = os.listdir(test_dir)
    test_images = []
    
    for filename in test_files:
        with open(os.path.join(test_dir, filename), 'rb') as f:
            test_images.append(transform(Image.open(f)))
    return torch.stack(test_images)
    
def create_train_and_valid_datasets(data):
    split_point = 9000
    train, valid = data[:split_point], data[split_point:]
    return train, valid
    
def get_training_dataset():
    clean_dir = os.path.join(data_dir, 'clean')
    distorted_dir = os.path.join(data_dir, 'distorted')
    #transforms.Lambda(lambda tensor: tensor.view(784))
    dataset = TrainDataset2(clean_dir, distorted_dir, 9000, transform)
    return dataset

def get_valid_dataset():
    clean_dir = os.path.join(data_dir, 'clean')
    distorted_dir = os.path.join(data_dir, 'distorted')
    #transforms.Lambda(lambda tensor: tensor.view(784))
    dataset = ValidDataset2(clean_dir, distorted_dir, 1000, transform)
    return dataset  

def get_testing_dataset():
    test_dir = os.path.join(data_dir, 'test_distorted')
    dataset = TestDataset2(test_dir, transform)
    return dataset

In [ ]:
def get_dataloaders(args, shuffle, clean, dist, test_data):    
    clean_train, clean_valid = create_train_and_valid_datasets(clean)
    dist_train, dist_valid = create_train_and_valid_datasets(dist)
    
    train_dataset = TrainDataset(clean_train, dist_train)
    valid_dataset = ValidDataset(clean_valid, dist_valid)
    test_dataset = TestDataset(test_data)
    
    train_loader = torch.utils.data.DataLoader(
            dataset=train_dataset,
            batch_size=args.train_batch_size,
            num_workers=1,
            shuffle=shuffle
    )
    valid_loader = torch.utils.data.DataLoader(
            dataset=valid_dataset,
            batch_size = args.train_batch_size,
            num_workers=1,
            shuffle=shuffle
    )
    test_loader = torch.utils.data.DataLoader(
            dataset=test_dataset,
            batch_size=args.test_batch_size,
            num_workers=1,
            shuffle=not shuffle)
    
    return train_loader, valid_loader, test_loader

 # Averaged metrics

In [ ]:
# EXPONENTIALLY DECAYED MOVING AVERAGE 
class AverageBase(object):
    def __init__(self, value=0):
        self.value = float(value) if value is not None else None
    
    def __str__(self):
        return str(round(self.value, 4))
    
    def __repr__(self):
        return self.value
    
    def __format__(self, fmt):
        return self.value.__format__(fmt)
    
    def __float__(self):
        return self.value
    

class RunningAverage(AverageBase):
    """
    Keeps track of a cumulative moving average (CMA).
    """
    
    def __init__(self, value=0, count=0):
        super(RunningAverage, self).__init__(value)
        self.count = count
        
    def update(self, value):
        self.value = (self.value * self.count + float(value))
        self.count += 1
        self.value /= self.count
        return self.value


class MovingAverage(AverageBase):
    """
    An exponentially decaying moving average (EMA).
    """
    
    def __init__(self, alpha=0.99):
        super(MovingAverage, self).__init__(None)
        self.alpha = alpha
        
    def update(self, value):
        if self.value is None:
            self.value = float(value)
        else:
            self.value = self.alpha * self.value + (1 - self.alpha) * float(value)
        return self.value

# Data preview

In [ ]:
def data_preview(dataset):
    plt.figure(figsize=(10,10))
    
    sample = dataset.train_data[:64]
    # shape (64, 28, 28)
    sample = sample.reshape(8,8,28,28)
    # shape (8, 8, 28, 28)
    sample = sample.permute(0,2,1,3)
    # shape (8, 28, 8, 28)
    sample = sample.reshape(8*28,8*28)
    # shape (8*28, 8*28)
    plt.imshow(sample)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title('64 samples from MNIST')
    plt.show()

    # print('Labels:', dataset.train_labels[:64].numpy())
def draw_after_epoch(samples, epoch):
    #print(len(samples), samples[0][0])
    w, h = 10, 10
    ax = []
    fig=plt.figure(figsize=(8, 8))
    rows, columns = 5, 4
    for i in range(1, columns*rows + 1):
        ax.append(fig.add_subplot(rows, columns, i))
        ax[-1].set_title(f"Epoch {epoch}, example {int((i-1)//2 + 1)}")
        idx = (i - 1) // 2
        img = samples[0][i%2][idx].permute(1,2,0)
        
        print(img.shape)
        plt.imshow(img.detach().numpy())
    plt.show()

# Checkpoints

In [ ]:
# UTILS

def save_checkpoint(optimizer, model, epoch, filename):
    checkpoint_dict = {
        'optimizer': optimizer.state_dict(),
        'model': model.state_dict(),
        'epoch': epoch
    }
    torch.save(checkpoint_dict, filename)


def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename)
    epoch = checkpoint_dict['epoch']
    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])
    return epoch

def draw_losses(train_losses, test_losses):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(10,6))
    plt.plot(epochs, train_losses, '-o', label='Training loss')
    plt.plot(epochs, test_losses, '-o', label='Validation loss')
    plt.legend()
    plt.title('Learning curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.xticks(epochs)
    plt.show()

# AE

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(48 * 48 * 3, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 48 * 48 * 3),
            nn.Sigmoid())
        self.classify = nn.Linear(64, 10)
    
    def forward(self, x):
        enc = self.encoder(x)
        dec = self.decoder(enc)
        return dec, enc #self.classify(enc)

# CONV AE

In [ ]:
from copy import deepcopy
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),  # b, 16, 10, 10
            nn.BatchNorm2d(16),
            nn.ELU(0.1),
        )
        self.enc2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.BatchNorm2d(32),
            nn.ELU(0.1),
        )
        self.enc3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ELU(0.1)
        )
        self.dec0 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 2, stride=2),
            nn.BatchNorm2d(32),
            nn.ELU(0.1)
        )
        self.dec1 = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 2, stride=2),  # b, 16, 5, 5
            nn.BatchNorm2d(16),
            nn.ELU(0.1),
        )
        self.dec2 = nn.Sequential(
            nn.ConvTranspose2d(16, 3, 2, stride=2),  # b, 8, 15, 15
            nn.BatchNorm2d(3),
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        x = self.enc1(x)
        x = self.enc2(x)
        enc = x
        x = self.dec1(x)
        dec = self.dec2(x)
        return dec, enc

# Better CAE

In [ ]:
class HugeConvAutoencoder(nn.Module):
    def __init__(self):
        super(HugeConvAutoencoder, self).__init__()
        self.enc_layer1 = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32,32,3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32,64,3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,64,3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2,2)
        )

        self.dec_layer2 = nn.Sequential(
            nn.ConvTranspose2d(64,64,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.ConvTranspose2d(64,32,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.ConvTranspose2d(32,32,3,2,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.ConvTranspose2d(32,3,3,2,1,1),
            nn.ReLU()
        )
        
    def forward(self, x):
        enc = self.enc_layer1(x)
        dec = self.dec_layer2(enc)
        return dec, enc

# Results

In [ ]:
import numpy as np

def save_result(images: np.ndarray, out_path: str):
    assert images.shape == (400, 3, 48, 48)
    
    flat_img = images.reshape(400, -1)
    n_rows = np.prod(images.shape)
    
    y_with_id = np.concatenate([np.arange(n_rows).reshape(-1, 1), flat_img.reshape(n_rows, 1)], axis=1)
    np.savetxt(out_path, y_with_id, delimiter=",", fmt=['%d', '%.4f'], header="id,expetced", comments='')

# Train & Test

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    train_loss = MovingAverage()
    criterion = nn.MSELoss()
    samples = []
    
    for batch_idx, (clean, distorted) in enumerate(train_loader):
        if batch_idx % 10 == 0: print(f"batch {batch_idx}")
        clean, distorted = clean.to(device), distorted.to(device)
        optimizer.zero_grad()
        output, _ = model(distorted)
        
        loss = criterion(clean, output)
        loss.backward()
        optimizer.step()        
        train_loss.update(loss)
    
    return samples, train_loss.value
    
def test(args, model, device, test_loader):
    criterion = nn.MSELoss()
    test_loss = RunningAverage()
    correct, all_examples = 0, 0
    samples = []
    all_loss = 0.
    with torch.no_grad():
        for batch_idx, (clean, distorted) in enumerate(test_loader):
            if batch_idx % 5 == 0: print(f"batch {batch_idx}")
            clean, distorted = clean.to(device), distorted.to(device)
            #clean = clean.view(clean.shape[0], -1)
            #distorted = distorted.view(distorted.shape[0], -1)
            output, classification = model(distorted)
            if batch_idx == 0:
                samples.append((distorted, output))
                #if batch_idx == 0: print(output[0])
                #samples.append((distorted.view(3,48,48), output.view(3,48,48)))
            loss = criterion(clean, output)
            all_loss += loss
            test_loss.update(loss)
    
    print(f"Summarized loss is {all_loss}")
    return samples, test_loss.value 

def get_test_results(data_loader, model):
    results = []
    with torch.no_grad():
        for batch_idx, distorted in enumerate(data_loader):
            distorted = distorted.to(device)
            output, _ = model(distorted)
            results.append(output)
    
    return torch.stack(results)
  
def main_loop(args, device, train_loader, val_loader, test_loader):
    #model = AutoEncoder().to(device)
    #model = ConvAutoencoder().to(device)
    model = HugeConvAutoencoder().to(device)
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-5)
    
    train_losses, test_losses = [], []
    min_loss = 1000.0
    for epoch in range(1, args.epochs + 1):
        print(f"Epoch {epoch}")
        _, train_loss = train(args, model, device, train_loader, optimizer, epoch)
        train_losses.append(train_loss)
        samples, test_loss = test(args, model, device, val_loader)
        test_losses.append(test_loss)
        if min_loss > test_loss:
            min_loss = test_loss
            torch.save(model.state_dict(), checkpoint_path)
            
        if not epoch % args.frequency:
            draw_after_epoch(samples, epoch)
    
    draw_losses(train_losses, test_losses)
    test_model = HugeConvAutoencoder().to(device)
    test_model.load_state_dict(torch.load(checkpoint_path))
    return get_test_results(test_loader, test_model)

# MAIN


In [ ]:
if __name__ == '__main__':    
    args = edict({
        'train_batch_size': 64,4/cAF29EwM0ZB76kbngNMhuN_tUZELv95NCX1B61iNkSrAPKNEagaOr5M
        'test_batch_size': 100,
        'epochs': 40,
        'lr': 0.0002,
        'momentum': 0.5, 
        'no_cuda': False,
        'valid_percent': 0.1,
        'seed': 7,
        'log_interval': 50,
        'save_model': False,
        'frequency': 3,
    })
    
#     clean_data, dist_data = get_clean_and_distorted_examples()
#     test_data = get_test_examples()
#     print(clean_data.shape, dist_data.shape, test_data.shape)
    clean_path = os.path.join(data_dir, 'clean.pt')
    dist_path = os.path.join(data_dir, 'dist.pt')
    test_path = os.path.join(data_dir, 'test.pt')
    
#     torch.save(clean_data, clean_path)
#     torch.save(dist_data, dist_path)
#     torch.save(test_data, test_path)
    clean_data = torch.load(clean_path)#, map_location={'cpu:', 'cuda:0'})
    dist_data = torch.load(dist_path)#, map_location=lambda storage, loc: storage.cuda(0))
    test_data = torch.load(test_path)#, map_location=lambda storage, loc: storage.cuda(0))
    print(clean_data.shape, dist_data.shape, test_data.shape)

    use_cuda = not args.no_cuda and torch.cuda.is_available()
    
    torch.manual_seed(args.seed)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    #train_loader, val_loader, test_loader = get_dataloaders(args, True, clean_data, dist_data, test_data)
    #test_results = main_loop(args, device, train_loader, val_loader, test_loader)
    test_results = test_results.reshape(400, 3, 48, 48)
    save_result(test_results.cpu().numpy(), results_path)

# Fast results generating

In [ ]:
checkpoint_path = os.path.join(data_dir, 'best_checkpoint.pkl')

test_path = os.path.join(data_dir, 'test.pt')
test_data = torch.load(test_path)

test_dataset = TestDataset(test_data)
    
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=100,
    num_workers=1,
    shuffle=False
)

test_model = HugeConvAutoencoder().to(device)
test_model.load_state_dict(torch.load(checkpoint_path))
test_results = get_test_results(test_loader, test_model)
test_results = test_results.reshape(400, 3, 48, 48)
save_result(test_results.cpu().numpy(), results_path)